In [48]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri
import numpy as np
from scipy.sparse import lil_matrix
import pandas as pd
import boto3, io, os

In [49]:
from tqdm import tqdm

In [50]:
transactions_df = pd.read_csv('transactions_sm.csv')
customers_df = pd.read_csv('customers_sm.csv')
articles_df = pd.read_csv('articles_sm.csv')

### 필요함 함수 목록

In [58]:
def get_most_bought_articles(data, num_articles=100):
    # Create dataframe that contains the number of times each article has been bought
    articles_counts = data[['article_id', 't_dat']].groupby('article_id').count().reset_index().rename(columns={'t_dat': 'count'})
    articles_counts = articles_counts.sort_values(by='count', ascending=False)
        
    most_bought_articles = articles_counts.loc[articles_counts['count'] >= num_articles]['article_id'].values
    
    return most_bought_articles

In [59]:
def preprocess_customers_df(customers_df, transactions_df):
    # Active 상태와 클럽 멤버 상태에 대하여 모두 Boolean처리
    customers_df[['Active']] = customers_df[['Active']].fillna(value=0) 
    customers_df['club_member_status'] = customers_df['club_member_status'] == 'ACTIVE'

    # 특정한 우편 번호에 대해서는 Binary Encoding을 수행한다.
    customers_df['common_postal_code'] = customers_df['postal_code'] == '2c29ae653a9282cce4151bd87643c907644e09541abc28ae87dea0d1f6603b1c'

    # 나이가 없는 user에 대하여는 평균 값을 사용하여 채운다.
    customers_df[['age']] = customers_df[['age']].fillna(value=customers_df['age'].mean())

    # 사용하지 않는 컬럼을 drop 한다(FN, 수신주기, 우편 번호).
    customers_df = customers_df.drop(['FN', 'fashion_news_frequency', 'postal_code'], axis=1)

    # Replace boolean with 1/0
    customers_df['club_member_status'] = customers_df['club_member_status'].astype(int)
    customers_df['common_postal_code'] = customers_df['common_postal_code'].astype(int)
    
    # 이너 조인 수행하여 구매 이력이 있는 고객만 남겨두었다
    customers_df = customers_df.merge(transactions_df[['customer_id', 'article_id']], on="customer_id")
    
    return customers_df

- 전체 고객 수
- training set의 unique 한 구매 고객 수

In [53]:
print(len(customers_df.customer_id))
print(len(customers_df.customer_id.unique()))

1371980
1371980


- training 기간 동안 transaction 이력이 있는 구매 고객

In [54]:
len(transactions_df.customer_id.unique())

445377

- validation data 생성하기
 - 공정한 test를 위해서 구매 이력이 있는 user에 대하여만 validation set을 구성한다.

In [55]:
most_bought_articles = get_most_bought_articles(transactions_df)

In [56]:
val_df = pd.read_csv('validation_sm.csv')

val_df = val_df[val_df.customer_id.isin(transactions_df.customer_id)]
#val_df = val_df.loc[val_df.article_id.isin(most_bought_articles)]
val_df = val_df[['customer_id', 'article_id']]
val_df['bought'] = np.ones(val_df.shape[0])

- 학습 대상이며 encoding을 해야할 고객 명단

In [57]:
customer_ids = transactions_df.customer_id.unique()
num_customers = len(customer_ids)
print(num_customers)

445377


- encoding을 해야할 상품 명단 (training 및 validation space에 포함된 모든 article에 대하여 encoding 한다)

In [172]:
article_ids = list(set(transactions_df.article_id.unique()).union(set(val_df.article_id.unique())))
num_articles = len(article_ids)
print(num_articles)

17394


In [173]:
# Create dictionaries with mapping keys
articles_id_to_idx = dict(zip(article_ids, range(num_articles)))
customers_id_to_idx = dict(zip(customer_ids, range(num_customers)))

In [174]:
train_df = transactions_df.copy()
train_df = train_df[['customer_id', 'article_id']]

num_transactions = train_df.shape[0]

train_df['bought'] = np.ones(num_transactions)

- negative sampling

In [175]:
np.random.seed(47)

negative_data = pd.DataFrame(
    {
        'article_id': np.random.permutation(train_df.article_id.values),
        'customer_id': train_df.customer_id.values,
        'bought': np.zeros(num_transactions)
    }
)

train_df = pd.concat([train_df, negative_data])
train_df = train_df.sample(frac=1).reset_index(drop=True)

- encoding 해주기
 - training encoding
 - validation encoding
 - customer encoding

In [176]:
train_df["customer_id"] = train_df["customer_id"].map(
        customers_id_to_idx
    )
train_df["article_id"] = train_df["article_id"].map(
    articles_id_to_idx
)

In [177]:
val_df["customer_id"] = val_df["customer_id"].map(
        customers_id_to_idx
    )
val_df["article_id"] = val_df["article_id"].map(
    articles_id_to_idx
)

In [191]:
customers_df['customer_id'] = customers_df["customer_id"].map(
        customers_id_to_idx
    )

In [200]:
customers_df.dropna(subset=['customer_id'],inplace=True)

## make a train and validation

- 전체 feature의 개수

In [179]:
nb_features=num_customers+num_articles + 4 # 추가된 numeric, binary features
nb_train=len(train_df.index)
nb_test=len(val_df.index)
print(f"전체 feature의 개수: {nb_features}")
print(f"전체 training data point: {nb_train}")
print(f"전체 test data point: {nb_test}")

전체 feature의 개수: 462775
전체 training data point: 5120954
전체 test data point: 178001


## Feature Engineering
 - 유저 Features : Age, Active, Club Member Status, Common Postal Codee
 - 상품 Features : 

In [185]:
def loadDataset(df, lines, columns,customers_df):
    # categorical feature는 one-hot encoding으로 numerical, binary feature들은 그대로 사용
    X = lil_matrix((lines, columns)).astype('float32')
    # Y 레이블은 np.numpy로 따로 저장
    Y = []
    # feature로 사용하고자 하는 customers의 데이터를
    customers_df = preprocess_customers_df(customers_df,df)
    line=0
    for index, row in tqdm(df.iterrows(),total=len(df), desc="Processing rows"):
            X[line,row['customer_id']] = 1
            X[line, num_customers+row['article_id']] = 1
            X[line, num_customers + num_articles] = customers_df['age'].iloc[line]
            X[line, num_customers + num_articles+1] = customers_df['Active'].iloc[line]
            X[line, num_customers + num_articles+2] = customers_df['club_member_status'].iloc[line]
            X[line, num_customers + num_articles+3] = customers_df['common_postal_code'].iloc[line]
            
            Y.append(row['bought'])
            line=line+1
            #assert line != 10
    Y=np.array(Y).astype('float32')            
    return X,Y

In [202]:
X_train, Y_train = loadDataset(train_df, nb_train, nb_features,customers_df)

Processing rows: 100%|██████████| 5120954/5120954 [18:21<00:00, 4647.48it/s]


In [201]:
X_test, Y_test = loadDataset(val_df, nb_test, nb_features,customers_df)

Processing rows: 100%|██████████| 178001/178001 [00:38<00:00, 4630.73it/s]


In [20]:
bucket = 'kanto.public'
prefix = 'inflab-extend'

if bucket.strip() == '':
    raise RuntimeError("bucket name is empty.")

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [18]:
%%time
def writeDatasetToProtobuf(X, bucket, prefix, key, d_type, Y=None):
    buf = io.BytesIO()
    if d_type == "sparse":
        smac.write_spmatrix_to_sparse_tensor(buf, X, labels=Y)
    else:
        smac.write_numpy_to_dense_tensor(buf, X, labels=Y)
        
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs


In [207]:
fm_train_data_path = writeDatasetToProtobuf(X_train, bucket, train_prefix, train_key, "sparse", Y_train)    
fm_test_data_path  = writeDatasetToProtobuf(X_test, bucket, test_prefix, test_key, "sparse", Y_test)    
  
print("Training data S3 path: ",fm_train_data_path)
print("Test data S3 path: ",fm_test_data_path)
print("FM model output S3 path: {}".format(output_prefix))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Training data S3 path:  s3://kanto.public/inflab-extend/train/train.protobuf
Test data S3 path:  s3://kanto.public/inflab-extend/test/test.protobuf
FM model output S3 path: s3://kanto.public/inflab-extend/output
CPU times: user 7min 2s, sys: 18.9 s, total: 7min 21s
Wall time: 6min 57s


In [208]:
#instance_type='ml.m5.2xlarge'
instance_type='ml.c5.4xlarge'
fm = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "factorization-machines"),
                                   get_execution_role(), 
                                   train_instance_count=4, 
                                   train_instance_type=instance_type,
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nb_features,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': 's3://kanto.public/inflab-extend/train/train.protobuf', 'test': fm_test_data_path},wait=False)

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [ ]:
fm.fit({'train': fm_train_data_path, 'test': fm_test_data_path},wait=False)

# Build KNN model 

In [2]:
!pip install mxnet

     |████████████████████████████████| 49.1 MB 96 kB/s              


In [7]:
import mxnet as mx
import os
model_file_name = "model.tar.gz"
model_full_path = "s3://kanto.public/inflab-extend/output/factorization-machines-2023-11-29-08-23-16-330/output/model.tar.gz"
#Download FM model 
os.system("aws s3 cp "+model_full_path+ " .")
os.system("tar xzvf "+model_file_name)
os.system("unzip -o model_algo-1")
os.system("mv symbol.json model-symbol.json")
os.system("mv params model-0000.params")

0

In [8]:
m = mx.module.Module.load('./model', 0, False, label_names=['out_label'])
V = m._arg_params['v'].asnumpy()
w = m._arg_params['w1_weight'].asnumpy()
b = m._arg_params['w0_weight'].asnumpy()

In [9]:
V.shape

(462775, 64)

In [10]:
w.shape

(462775, 1)

In [15]:
# item latent matrix - concat(V[i], w[i]).  
knn_item_matrix = np.concatenate((V[445377:445377+17394], w[445377:445377+17394]), axis=1)
knn_train_label = np.arange(17394)

#user latent matrix - concat (V[u], 1) 
ones = np.ones(445377).reshape((445377, 1))
knn_user_matrix = np.concatenate((V[:445377], ones), axis=1)

In [21]:
print('KNN train features shape = ', knn_item_matrix.shape)
bucket = 'kanto.public'
knn_prefix = 'inflab-extend/knn'
knn_output_prefix  = 's3://{}/{}/output'.format(bucket, knn_prefix)
knn_train_data_path = writeDatasetToProtobuf(knn_item_matrix, bucket, knn_prefix, train_key, "dense", knn_train_label)
print('uploaded KNN train data: {}'.format(knn_train_data_path))

KNN train features shape =  (17394, 65)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


uploaded KNN train data: s3://kanto.public/inflab-extend/knn/train.protobuf


In [83]:
knn_item_matrix[0]

array([-1.02020393e-03, -6.15481753e-04,  1.69058028e-03,  1.41442684e-03,
        4.81207680e-04,  1.08488915e-04, -5.61226218e-04,  6.70878871e-05,
        6.77982578e-04, -2.37829183e-04, -8.49267293e-04,  1.26664600e-04,
        3.85732739e-04, -1.83520126e-04,  2.16518965e-04,  7.43333250e-04,
        1.17466634e-03, -1.18313951e-03, -6.96834933e-04,  1.02745078e-03,
        1.08534913e-03,  2.01333733e-05, -2.12530326e-03,  6.78313721e-04,
        5.43291273e-04,  3.62173421e-04,  7.63809949e-05,  1.07053563e-03,
       -6.00218598e-04,  1.16842543e-03, -3.51635332e-04,  1.76417804e-03,
        2.96203914e-04,  2.31795688e-03, -9.85596329e-04, -3.58214951e-04,
        1.03862642e-03,  6.81627425e-04,  2.93109246e-04, -6.58792735e-04,
       -8.23677576e-04,  3.16859660e-04, -1.66957651e-03,  1.14991017e-04,
        8.22708185e-04,  4.83559299e-04,  2.03278172e-03,  3.96121555e-04,
       -1.29694073e-03,  2.70962453e-04,  2.25974145e-04,  1.77530746e-03,
       -8.99909355e-04,  

In [22]:
nb_recommendations = 12
instance_type='ml.c5.4xlarge'
# set up the estimator
knn = sagemaker.estimator.Estimator(get_image_uri(boto3.Session().region_name, "knn"),
    get_execution_role(),
    train_instance_count=1,
    train_instance_type=instance_type,
    output_path=knn_output_prefix,
    sagemaker_session=sagemaker.Session())

knn.set_hyperparameters(feature_dim=knn_item_matrix.shape[1], k=nb_recommendations, index_metric="INNER_PRODUCT", predictor_type='classifier', sample_size=10000)
fit_input = {'train': knn_train_data_path}
knn.fit(fit_input,wait=False)
knn_model_name =  knn.latest_training_job.job_name

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [63]:
knn.deploy(instance_type='ml.m5.large',initial_instance_count=1)

---------!

In [66]:
import sagemaker
from sagemaker.predictor import RealTimePredictor
from sagemaker import get_execution_role

In [72]:
predictor = sagemaker.KNNPredictor(endpoint_name='knn-2023-11-30-08-16-33-857', sagemaker_session=sagemaker.Session())

In [85]:
predictor.predict(data=knn_user_matrix[1424])

[label {
   key: "predicted_label"
   value {
     float64_tensor {
       values: 1575.0
     }
   }
 }]